In [1]:
!pip install nltk

In [2]:
!pip install wordcloud

In [3]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
import tensorflow
import tensorflow as tf

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, classification_report
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import Model
from keras.models import Sequential
from keras import models, layers, optimizers, regularizers
from keras.layers import Dense, Embedding, MaxPool1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU, Bidirectional, Dropout, MaxPooling1D, Input, Lambda, GlobalMaxPool1D
from keras.callbacks import ModelCheckpoint
from collections import Counter
from keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
tensorflow.random.set_seed(42)

In [5]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [7]:
LEMMATIZER = WordNetLemmatizer()
PORTER_STEMMER = PorterStemmer()
ENGLISH_STOP_WORDS = set(stopwords.words('english'))

In [8]:
# from google.colab import drive

In [9]:
# drive.mount('/content/drive')

In [10]:
DF_ORIG = pd.read_csv("/kaggle/input/unit3-train/train.csv", index_col=0)

In [11]:
DF_ORIG.head()

,Text,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [12]:
DF_ORIG.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41159 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       41158 non-null  object
 1   Sentiment  41155 non-null  object
dtypes: object(2)
memory usage: 964.7+ KB


In [13]:
DF_ORIG.describe()

,Text,Sentiment
count,41158,41155
unique,41158,5
top,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Positive
freq,1,11422


### Preprocessing

In [14]:
def text_preprocessing(text, stop_words=None, stem=None, lemmatizer=None):
    if isinstance(text, str):
        text = re.sub(r"http\S+", "", text)

    text = text.translate(str.maketrans("", "", string.punctuation))
    text = "".join([i for i in text if not i.isdigit()])
    text = text.lower()

    if stop_words is not None:
        words = nltk.word_tokenize(text)
        filtered_words = [word for word in words if word not in stop_words]
        text = " ".join(filtered_words)

    if stem is not None:
        words = nltk.word_tokenize(text)
        stemmed_words = [stem.stem(word) for word in words]
        text = " ".join(stemmed_words)

    if lemmatizer is not None:
        words = nltk.word_tokenize(text)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        text = " ".join(lemmatized_words)

    return text

In [15]:
DF_ORIG.isna().sum().sum()

5

In [16]:
DF_ORIG.dropna(inplace=True)

In [17]:
DF_ORIG.isna().sum().sum()

0

In [18]:
DF_ORIG["Text"] = DF_ORIG["Text"].apply(text_preprocessing, args=(ENGLISH_STOP_WORDS, PORTER_STEMMER, LEMMATIZER,))

In [19]:
DF_ORIG = DF_ORIG[DF_ORIG["Text"] != ""]

In [20]:
DF_ORIG.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41139 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       41139 non-null  object
 1   Sentiment  41139 non-null  object
dtypes: object(2)
memory usage: 964.2+ KB


In [21]:
DF_ORIG.head()

,Text,Sentiment
0,menyrbi philgahan chrisitv,Neutral
1,advic talk neighbour famili exchang phone numb...,Positive
2,coronaviru australia woolworth give elderli di...,Positive
3,food stock one empti plea dont panic enough fo...,Positive
4,readi go supermarket covid outbreak im paranoi...,Extremely Negative


### Tokenization

In [22]:
df_train = DF_ORIG.copy()

In [23]:
df_train.shape

(41139, 2)

In [24]:
counter = Counter()

In [25]:
VOC_LEN = len(df_train.Text.apply(counter.update))
VOC_LEN

41139

In [26]:
tokenizer = Tokenizer(num_words=VOC_LEN)

In [27]:
tokenizer.fit_on_texts(df_train.Text)

In [28]:
X_vect = tokenizer.texts_to_sequences(df_train.Text)

In [29]:
maxlen = len(max(X_vect, key=len))

In [30]:
X_vect_pads = pad_sequences(X_vect, maxlen=maxlen)

In [31]:
X_vect_pads

array([[    0,     0,     0, ..., 17378, 17379,  9380],
       [    0,     0,     0, ...,   972,  2192,    84],
       [    0,     0,     0, ...,   103,     1,    75],
       ...,
       [    0,     0,     0, ...,  1227,    25,    19],
       [    0,     0,     0, ...,     2,     1,     2],
       [    0,     0,     0, ...,   280,   709,     1]], dtype=int32)

In [32]:
label_encoder = LabelEncoder()
y_vect = label_encoder.fit_transform(df_train.Sentiment)
y_vect = to_categorical(y_vect)
y_vect

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_vect_pads, y_vect, test_size=0.25, random_state=42)

In [34]:
LOGITS_QTY = len(df_train["Sentiment"].unique())

In [35]:
LOGITS_QTY

5

### LSTM variations

#### 1.

In [36]:
model = Sequential([
    Embedding(input_dim=VOC_LEN, output_dim=256, input_length=maxlen),
    LSTM(256, return_sequences=True),
    Dropout(0.5),
    LSTM(32),
    Dense(LOGITS_QTY, activation='softmax')
])

In [37]:
model.compile(optimizer=Adam(learning_rate=0.005), loss="categorical_crossentropy", metrics=["accuracy"])

In [38]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
965/965 [==============================] - 42s 38ms/step - loss: 1.0319 - accuracy: 0.5846 - val_loss: 0.8458 - val_accuracy: 0.6807
Epoch 2/5
965/965 [==============================] - 13s 13ms/step - loss: 0.7189 - accuracy: 0.7349 - val_loss: 0.8584 - val_accuracy: 0.6766
Epoch 3/5
965/965 [==============================] - 11s 12ms/step - loss: 0.5674 - accuracy: 0.8017 - val_loss: 0.9031 - val_accuracy: 0.6701
Epoch 4/5
965/965 [==============================] - 11s 11ms/step - loss: 0.4695 - accuracy: 0.8398 - val_loss: 0.9835 - val_accuracy: 0.6755
Epoch 5/5
965/965 [==============================] - 10s 10ms/step - loss: 0.4076 - accuracy: 0.8651 - val_loss: 1.0278 - val_accuracy: 0.6652


#### 2.

In [39]:
model1 = Sequential([
    Embedding(input_dim=VOC_LEN, output_dim=128, input_length=maxlen),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(LOGITS_QTY, activation='softmax')
])

In [40]:
model1.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

In [41]:
model1.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/5
483/483 [==============================] - 42s 74ms/step - loss: 1.2034 - accuracy: 0.4876 - val_loss: 0.9280 - val_accuracy: 0.6317
Epoch 2/5
483/483 [==============================] - 13s 27ms/step - loss: 0.7300 - accuracy: 0.7294 - val_loss: 0.8458 - val_accuracy: 0.6770
Epoch 3/5
483/483 [==============================] - 9s 18ms/step - loss: 0.5421 - accuracy: 0.8124 - val_loss: 0.8767 - val_accuracy: 0.6712
Epoch 4/5
483/483 [==============================] - 9s 18ms/step - loss: 0.4230 - accuracy: 0.8605 - val_loss: 0.9953 - val_accuracy: 0.6654
Epoch 5/5
483/483 [==============================] - 9s 18ms/step - loss: 0.3416 - accuracy: 0.8934 - val_loss: 1.0732 - val_accuracy: 0.6643


### Gru variations

#### 1.

In [42]:
gru = Sequential([
    Embedding(input_dim=VOC_LEN, output_dim=128, input_length=maxlen),
    GRU(128, return_sequences=False),
    Dropout(0.25),
    Dense(LOGITS_QTY, activation='softmax')
])

In [43]:
gru.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
gru.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/4
965/965 [==============================] - 35s 34ms/step - loss: 1.0638 - accuracy: 0.5598 - val_loss: 0.8191 - val_accuracy: 0.6876
Epoch 2/4
965/965 [==============================] - 10s 10ms/step - loss: 0.6653 - accuracy: 0.7555 - val_loss: 0.8056 - val_accuracy: 0.6942
Epoch 3/4
965/965 [==============================] - 8s 8ms/step - loss: 0.4630 - accuracy: 0.8377 - val_loss: 0.9125 - val_accuracy: 0.6676
Epoch 4/4
965/965 [==============================] - 7s 7ms/step - loss: 0.3122 - accuracy: 0.8920 - val_loss: 1.0431 - val_accuracy: 0.6624


#### 2.

In [45]:
gru1 = Sequential([
    Embedding(input_dim=VOC_LEN, output_dim=128, input_length=maxlen),
    Bidirectional(GRU(128, return_sequences=True)),
    Bidirectional(GRU(128)),
    Dropout(0.25),
    Dense(LOGITS_QTY, activation='softmax')
])

In [46]:
gru1.compile(optimizer=Adam(learning_rate=0.002), loss='mean_squared_error', metrics=['accuracy'])

In [47]:
gru1.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/4
965/965 [==============================] - 41s 35ms/step - loss: 0.1103 - accuracy: 0.5739 - val_loss: 0.0884 - val_accuracy: 0.6844
Epoch 2/4
965/965 [==============================] - 17s 17ms/step - loss: 0.0742 - accuracy: 0.7409 - val_loss: 0.0875 - val_accuracy: 0.6930
Epoch 3/4
965/965 [==============================] - 16s 16ms/step - loss: 0.0515 - accuracy: 0.8298 - val_loss: 0.0958 - val_accuracy: 0.6787
Epoch 4/4
965/965 [==============================] - 15s 16ms/step - loss: 0.0362 - accuracy: 0.8851 - val_loss: 0.0998 - val_accuracy: 0.6825
